In [1]:
# Importamos pandas y numpy
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
#Expecificamos ruta

url = r'https://raw.githubusercontent.com/cdeniaca/Group-7/76322e4482f3f9e2d42538a322aeb517e693a958/Poblacion_Activa.csv'

In [4]:
#Cargamos el df

población_activa_df = pd.read_csv(url, sep=';')


In [ ]:
#Imprimimos el df

print(población_activa_df)

In [ ]:
# Muestra los tipos de datos de cada columna
print(población_activa_df.dtypes)


In [7]:
# Reemplazamos el punto por una cadena vacía para eliminar el separador de miles
población_activa_df['Total'] = población_activa_df['Total'].str.replace('.', '', regex=False)

# Reemplazamos la coma por un punto para que pandas pueda interpretarlo como decimal
población_activa_df['Total'] = población_activa_df['Total'].str.replace(',', '.', regex=False)

In [ ]:
# Convertimos la columna a tipo numérico
población_activa_df['Total'] = pd.to_numeric(población_activa_df['Total'], errors='coerce')

# Verificamos el resultado
print(población_activa_df['Total'].head())


In [9]:
# Asegúrate de que la columna 'Total' exista en tu DataFrame
if 'Total' in población_activa_df.columns:
    población_activa_df['Total'] = población_activa_df['Total'] * 1000
else:
    print("La columna 'Total' no se encuentra en el DataFrame.")

In [10]:
# Crear una columna 'AÑO' con los primeros 4 caracteres de 'Periodo'
población_activa_df['AÑO'] = población_activa_df['Periodo'].str[:4]

# Crear una columna 'Trimestre' con los caracteres restantes de 'Periodo'
población_activa_df['Trimestre'] = población_activa_df['Periodo'].str[4:]

# Crear una columna 'Trimestre' con los caracteres restantes de 'Tip'



In [11]:
# Filtrar el DataFrame para que solo contenga los años de 2018 a 2023
años_filtrados = ['2018', '2019', '2020', '2021', '2022', '2023']
población_activa_df = población_activa_df[
    población_activa_df['AÑO'].isin(años_filtrados) &
    (población_activa_df['Comunidad autónoma'] != 'Total Nacional') &
    (población_activa_df['Relación con la actividad en el trimestre actual'] != 'Total (trimestre actual)') &
    (población_activa_df['Relación con la actividad en el trimestre anterior'] == 'Total (trimestre anterior)')
]

In [12]:
# Usa apply() con una función lambda para separar y quedarte solo con la segunda parte de la cadena
población_activa_df['Comunidad autónoma'] = población_activa_df['Comunidad autónoma'].apply(lambda x: x.split(' ', 1)[1] if ' ' in x else x)


In [ ]:
población_activa_df

In [ ]:
# Contar las filas duplicadas
duplicados = población_activa_df.duplicated().sum()
print(f"Número de filas duplicadas: {duplicados}")


In [ ]:
# Contar los valores nulos en cada columna
nulos = población_activa_df.isnull().sum()
print("Número de valores nulos en cada columna:")
print(nulos)


In [ ]:
# Filtrar las filas que tienen al menos un valor nulo
filas_nulas = población_activa_df[población_activa_df.isnull().any(axis=1)]

# Mostrar las filas con valores nulos
print("Filas con valores nulos:")
print(filas_nulas)



In [ ]:
print(población_activa_df['Comunidad autónoma'].unique())


In [ ]:
print(población_activa_df['Relación con la actividad en el trimestre actual'].unique())


In [ ]:
# Comprobar duplicados en las combinaciones de índice
duplicados = población_activa_df.duplicated(subset=['Periodo', 'Comunidad autónoma', 'AÑO', 'Trimestre', 'Relación con la actividad en el trimestre actual'], keep=False)
if duplicados.any():
    print("Hay duplicados en los datos:")
    print(población_activa_df[duplicados])
else:
    print("No hay duplicados, puedes usar pivot() sin problemas.")


In [20]:
# Eliminar duplicados
población_activa_df = población_activa_df.drop_duplicates(subset=['Periodo', 'Comunidad autónoma', 'AÑO', 'Trimestre', 'Relación con la actividad en el trimestre actual'])


In [ ]:
print(población_activa_df['Relación con la actividad en el trimestre actual'].unique())


In [22]:
# Crear un DataFrame pivotado
df_pivot = pd.pivot_table(
    población_activa_df,
    index=['Periodo', 'Comunidad autónoma', 'AÑO', 'Trimestre'],  # Índices de filas
    columns='Relación con la actividad en el trimestre actual',  # Nuevas columnas
    values='Total',  # Valores a pivotar
    aggfunc='sum',  # Función de agregación
    fill_value=0  # Opcional: reemplaza valores NaN por 0
)


In [23]:
# Restablecer el índice para convertirlo en columnas regulares
df_pivot_reset = df_pivot.reset_index()

In [ ]:
# Imprimir las columnas para verificar los nombres
print(df_pivot_reset.columns)

In [25]:
df_pivot = df_pivot_reset

In [ ]:
df_pivot

In [27]:
# Renombrar las columnas específicas en el DataFrame
df_pivot = df_pivot.rename(columns={
    'Inactivos (trimestre actual)': 'Inactivos',
    'Ocupados (trimestre actual)': 'Ocupados',
    'Parados (trimestre actual)': 'Parados'
})

In [28]:
# Crear la nueva columna "Total Actividad" 
df_pivot['Total Actividad'] = (
    df_pivot['Inactivos'].fillna(0) +  # Considerar nulos como 0
    df_pivot['Ocupados'].fillna(0) +   # Considerar nulos como 0
    df_pivot['Parados'].fillna(0)      # Considerar nulos como 0
)


In [ ]:
df_pivot

In [30]:
#Dataframe Ocupados

In [ ]:
print(df_pivot.columns)


In [32]:
# Seleccionar las columnas deseadas de df_pivot
df_nuevo = df_pivot[['Comunidad autónoma', 'AÑO', 'Ocupados', 'Parados', 'Inactivos', 'Total Actividad']]



In [ ]:
df_nuevo

In [34]:
# Agrupar por "AÑO" y "Comunidad autónoma" y calcular la media de "Ocupados" y "Total Actividad"
df_nuevo_agrupado = df_nuevo.groupby(['AÑO', 'Comunidad autónoma']).mean().reset_index()



In [ ]:
df_nuevo_agrupado

In [36]:
df_nuevo_agrupado.to_csv('Poblacion_Ocupada_Agrupado.csv', index=False)

In [37]:
df_pivot.to_csv('Poblacion_Activa_Limpia.csv', index=False)

_____________________________________________________________________________________________________________________________________________________________________________________________

############################################## NUEVO CSV - TELETRABAJO ########################################################

In [38]:
#Expecificamos ruta

url_teletrabajo = r'https://raw.githubusercontent.com/cdeniaca/Group-7/refs/heads/main/Poblacion_Teletrabajo.csv'

In [39]:
#Cargamos el df

población_teletrabajo_df = pd.read_csv(url_teletrabajo, sep=';')

In [40]:
# Reemplazamos el punto por una cadena vacía para eliminar el separador de miles
población_teletrabajo_df['Total'] = población_teletrabajo_df['Total'].str.replace('.', '', regex=False)

# Reemplazamos la coma por un punto para que pandas pueda interpretarlo como decimal
población_teletrabajo_df['Total'] = población_teletrabajo_df['Total'].str.replace(',', '.', regex=False)

In [41]:
# Convertimos la columna a tipo numérico
población_teletrabajo_df['Total'] = pd.to_numeric(población_teletrabajo_df['Total'], errors='coerce')


In [42]:
población_teletrabajo_df['Comunidades y Ciudades Autónomas'] = población_teletrabajo_df['Comunidades y Ciudades Autónomas'].apply(
    lambda x: x[3:]
)


In [ ]:
# Muestra los tipos de datos de cada columna
print(población_teletrabajo_df.dtypes)

In [44]:
# Elimina filas donde 'Total' tenga NaN
población_teletrabajo_df = población_teletrabajo_df.dropna(subset=['Total'])


In [45]:
población_teletrabajo_df.rename(columns={'Comunidades y Ciudades Autónomas': 'Comunidad autónoma'}, inplace=True)
población_teletrabajo_df.rename(columns={'Año': 'AÑO'}, inplace=True)

In [ ]:
print(población_teletrabajo_df['Comunidad autónoma'].unique())


In [ ]:
print(población_teletrabajo_df['Teletrabajo'].unique())

In [ ]:
print(población_teletrabajo_df['AÑO'].unique())

In [49]:
# Crear un DataFrame pivotado
df_pivot_teletrabajo = pd.pivot_table(
    población_teletrabajo_df,
    index=['AÑO', 'Comunidad autónoma'],  # Asegúrate de usar el nombre correcto
    columns='Teletrabajo',
    values='Total',
    aggfunc='sum',
    fill_value=0
)


In [50]:
# Restablecer el índice para convertirlo en columnas regulares
df_pivot_teletrabajo = df_pivot_teletrabajo.reset_index()

In [51]:
# Multiplicación y división para calcular valores absolutos basados en porcentajes
df_pivot_teletrabajo['Han teletrabajado'] = df_pivot_teletrabajo['Han teletrabajado'] * df_pivot_teletrabajo['Total'] / 100
df_pivot_teletrabajo['No han teletrabajado aunque su trabajo les permitiría teletrabajar'] = df_pivot_teletrabajo['No han teletrabajado aunque su trabajo les permitiría teletrabajar'] * df_pivot_teletrabajo['Total'] / 100
df_pivot_teletrabajo['No han teletrabajado porque su trabajo no les permitiría teletrabajar'] = df_pivot_teletrabajo['No han teletrabajado porque su trabajo no les permitiría teletrabajar'] * df_pivot_teletrabajo['Total'] / 100

In [52]:
df_pivot_teletrabajo = df_pivot_teletrabajo[['AÑO', 'Comunidad autónoma', 'Han teletrabajado', 'No han teletrabajado aunque su trabajo les permitiría teletrabajar', 'No han teletrabajado porque su trabajo no les permitiría teletrabajar']]


In [53]:
# Dividir las columnas seleccionadas entre 1000 para cambiar de millones a miles
columnas_a_convertir = [
    'No han teletrabajado aunque su trabajo les permitiría teletrabajar',
    'Han teletrabajado',
    'No han teletrabajado porque su trabajo no les permitiría teletrabajar'
]

# Realizar la conversión
df_pivot_teletrabajo[columnas_a_convertir] = df_pivot_teletrabajo[columnas_a_convertir] / 1000


In [54]:
media_por_año = df_pivot_teletrabajo.groupby('AÑO')[[
    'No han teletrabajado aunque su trabajo les permitiría teletrabajar',
    'Han teletrabajado',
    'No han teletrabajado porque su trabajo no les permitiría teletrabajar'
]].mean().reset_index()

In [55]:
# Convertir la columna 'AÑO' a tipo int en ambos DataFrames, o cambiar a str si prefieres.
df_nuevo_agrupado['AÑO'] = df_nuevo_agrupado['AÑO'].astype(int)
df_pivot_teletrabajo['AÑO'] = df_pivot_teletrabajo['AÑO'].astype(int)

# O convertir ambas a string
# df_nuevo_agrupado['AÑO'] = df_nuevo_agrupado['AÑO'].astype(str)
# población_teletrabajo_df['AÑO'] = población_teletrabajo_df['AÑO'].astype(str)


In [56]:
df_final_teletrabajo = pd.merge(df_nuevo_agrupado, df_pivot_teletrabajo, on=['Comunidad autónoma', 'AÑO'], how='inner')


In [ ]:
df_final_teletrabajo


_________________________________________________________________________________________________________________________________________________________________________________

################################################### CSV - GASTOS TRANSPORTES ###################################################

_________________________________________________________________________________________________________________________________________________________________________________

In [58]:
#Expecificamos ruta gastos

url_gastos = r'https://raw.githubusercontent.com/cdeniaca/Group-7/refs/heads/main/Gastos_Transporte_Limpio.csv'

In [59]:
gasto_transporte_df = pd.read_csv(url_gastos,  sep=',')

In [ ]:
gasto_transporte_df

In [61]:
# Filtrar para excluir "Ceuta y Melilla" en la columna "Comunidad autónoma" usando query
df_nuevo_agrupado_filtrado = df_nuevo_agrupado.query("`Comunidad autónoma` != 'Ceuta y Melilla'")

In [62]:
# Convertir la columna 'AÑO' a tipo int en ambos DataFrames, o cambiar a str si prefieres.
df_nuevo_agrupado_filtrado['AÑO'] = df_nuevo_agrupado_filtrado['AÑO'].astype(int)
gasto_transporte_df['Periodo'] = gasto_transporte_df['Periodo'].astype(int)

In [63]:
gasto_transporte_df.rename(columns={'Comunidades_Autonomas': 'Comunidad autónoma'}, inplace=True)
gasto_transporte_df.rename(columns={'Periodo': 'AÑO'}, inplace=True)
gasto_transporte_df.rename(columns={'Total': 'Gastos en transporte unitarios'}, inplace=True)

In [64]:
df_final_gastos = pd.merge(df_nuevo_agrupado_filtrado, gasto_transporte_df, on=['Comunidad autónoma', 'AÑO'], how='inner')

In [65]:
df_final_gastos['Gasto total en transporte'] = df_final_gastos['Ocupados'] * df_final_gastos['Gastos en transporte unitarios']


In [66]:
df_final_gastos = df_final_gastos.drop(columns=['Tipo_Coste'])


In [ ]:
df_final_gastos

_________________________________________________________________________________________________________________________________________________________________________________

################################################### CSV - SALARIOS ###################################################

_________________________________________________________________________________________________________________________________________________________________________________

In [102]:
#Expecificamos ruta salarios

url_salarios = r'https://raw.githubusercontent.com/cdeniaca/Group-7/refs/heads/main/Salarios_Limpio.csv'

In [103]:
salarios_df = pd.read_csv(url_salarios,  sep=',')

In [104]:
salarios_df['Periodo'] = salarios_df['Periodo'].astype(int)


In [105]:
# Convertir la columna 'Total'
salarios_df['Total'] = (
    salarios_df['Total']
    .astype(str)
    .str.replace('.', '', regex=False)  # Quitar el punto
    .str.replace(',', '.', regex=False)  # Cambiar la coma por punto
    .astype(float)  # Convertir a float
)

In [107]:
salarios_df.rename(columns={'Comunidades_Autonomas': 'Comunidad autónoma'}, inplace=True)
salarios_df.rename(columns={'Periodo': 'AÑO'}, inplace=True)
salarios_df.rename(columns={'Total': 'Sueldos y salarios unitarios'}, inplace=True)

In [108]:
df_final_salarios= pd.merge(df_nuevo_agrupado_filtrado, salarios_df, on=['Comunidad autónoma', 'AÑO'], how='inner')

In [109]:
df_final_salarios['Sueldos y salarios totales'] = df_final_salarios['Ocupados'] * df_final_salarios['Sueldos y salarios unitarios']

In [110]:
df_final_salarios = df_final_salarios.drop(columns=['Tipo_Coste'])

In [111]:
df_final_salarios

,AÑO,Comunidad autónoma,Ocupados,Parados,Inactivos,Total Actividad,Sector,Sueldos y salarios unitarios,Sueldos y salarios totales
0,2018,Andalucía,"3,030,750.00","904,625.00","2,983,100.00","6,918,475.00",Industria,"24,335.01","73,753,331,557.50"
1,2018,Andalucía,"3,030,750.00","904,625.00","2,983,100.00","6,918,475.00",Construcción,"20,853.79","63,202,624,042.50"
2,2018,Andalucía,"3,030,750.00","904,625.00","2,983,100.00","6,918,475.00",Servicios,"19,438.66","58,913,718,795.00"
3,2018,Aragón,"573,825.00","68,250.00","452,275.00","1,094,350.00",Industria,"26,716.39","15,330,532,491.75"
4,2018,Aragón,"573,825.00","68,250.00","452,275.00","1,094,350.00",Construcción,"23,471.18","13,468,349,863.50"
...,...,...,...,...,...,...,...,...,...
301,2023,País Vasco,"995,750.00","83,375.00","818,725.00","1,897,850.00",Construcción,"29,392.81","29,267,890,557.50"
302,2023,País Vasco,"995,750.00","83,375.00","818,725.00","1,897,850.00",Servicios,"28,236.98","28,116,972,835.00"
303,2023,"Rioja, La","146,625.00","15,150.00","112,600.00","274,375.00",Industria,"25,845.14","3,789,543,652.50"
304,2023,"Rioja, La","146,625.00","15,150.00","112,600.00","274,375.00",Construcción,"25,321.95","3,712,830,918.75"


In [112]:
df_final_gst_salario= pd.merge(df_final_gastos, df_final_salarios, on=['Comunidad autónoma', 'AÑO', 'Sector', 'Ocupados', 'Parados', 'Inactivos', 'Total Actividad'], how='inner')

In [113]:
df_final_gst_salario

,AÑO,Comunidad autónoma,Ocupados,Parados,Inactivos,Total Actividad,Sector,Gastos en transporte unitarios,Gasto total en transporte,Sueldos y salarios unitarios,Sueldos y salarios totales
0,2018,Andalucía,"3,030,750.00","904,625.00","2,983,100.00","6,918,475.00",Industria,37.15,"112,592,362.50","24,335.01","73,753,331,557.50"
1,2018,Andalucía,"3,030,750.00","904,625.00","2,983,100.00","6,918,475.00",Construcción,21.25,"64,403,437.50","20,853.79","63,202,624,042.50"
2,2018,Andalucía,"3,030,750.00","904,625.00","2,983,100.00","6,918,475.00",Servicios,3.39,"10,274,242.50","19,438.66","58,913,718,795.00"
3,2018,Aragón,"573,825.00","68,250.00","452,275.00","1,094,350.00",Industria,109.99,"63,115,011.75","26,716.39","15,330,532,491.75"
4,2018,Aragón,"573,825.00","68,250.00","452,275.00","1,094,350.00",Construcción,5.97,"3,425,735.25","23,471.18","13,468,349,863.50"
...,...,...,...,...,...,...,...,...,...,...,...
301,2023,País Vasco,"995,750.00","83,375.00","818,725.00","1,897,850.00",Construcción,30.82,"30,689,015.00","29,392.81","29,267,890,557.50"
302,2023,País Vasco,"995,750.00","83,375.00","818,725.00","1,897,850.00",Servicios,7.54,"7,507,955.00","28,236.98","28,116,972,835.00"
303,2023,"Rioja, La","146,625.00","15,150.00","112,600.00","274,375.00",Industria,16.48,"2,416,380.00","25,845.14","3,789,543,652.50"
304,2023,"Rioja, La","146,625.00","15,150.00","112,600.00","274,375.00",Construcción,11.38,"1,668,592.50","25,321.95","3,712,830,918.75"


In [114]:
df_final_gst_salario.to_csv('Poblacion_Salarios_Gastos.csv', index=False)